In [3]:
%run nfl-parse-demo.py

Name, Major:
0, http://www.steelers.com/cda-web/rss-module.htm?tagName=News
1, http://www.steelers.com/cda-web/rss-module.htm?tagName=Photos
2, http://www.steelers.com/cda-web/rss-module.htm?tagName=Videos
3, http://www.steelers.com/cda-web/rss-module.htm?tagName=Audio
4, http://www.steelers.com/cda-web/rss-module.htm?tagName=Espa%C3%B1ol
5, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News
6, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Press%20Releases
7, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Transcripts
8, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Late%20For%20Work
9, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20John%20Eisenberg
10, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20The%20Caw
11, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Byrne%20Identity
12, http://www.baltimoreravens.com/cda-w

In [4]:
vals

[['0',
  'Steelers',
  'News',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=News',
  'Steelers.com'],
 ['1',
  'Steelers',
  'Photos',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Photos',
  'Steelers.com'],
 ['2',
  'Steelers',
  'Videos',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Videos',
  'Steelers.com'],
 ['3',
  'Steelers',
  'Audio',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Audio',
  'Steelers.com'],
 ['4',
  'Steelers',
  'Espanol',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Espa%C3%B1ol',
  'Steelers.com'],
 ['5',
  'Ravens',
  'All News',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News'],
 ['6',
  'Ravens',
  'Press Releases',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Press%20Releases'],
 ['7',
  'Ravens',
  'Transcripts',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Transcripts'],
 ['8',
  'Ravens',
  'Late For Work'

In [2]:
import requests
#from bokeh.models import ColumnDataSource
from dateutil.parser import parse
from datetime import datetime as dt
import pandas as pd

In [3]:
api_url = 'http://www.bengals.com/cda-web/rss-module.htm?tagName=News%20Stories'

session = requests.Session()
session.mount('http://', requests.adapters.HTTPAdapter(max_retries=3))
raw_data = session.get(api_url)

# Parse to Initial DataFrame
#test_read = pd.read_json(raw_data.text, typ='series')

In [4]:
xml_data = raw_data.text


In [5]:
import xml.etree.ElementTree as ET

In [6]:
def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for subchild in child:
            record[subchild.tag] = subchild.text
            all_records.append(record)
    return pd.DataFrame(all_records)

In [18]:
xml2df(xml_data).head()

,description,item,link,title
0,News from the Cincinnati Bengals,\n,http://www.bengals.com,Cincinnati Bengals : News Stories
1,News from the Cincinnati Bengals,\n,http://www.bengals.com,Cincinnati Bengals : News Stories
2,News from the Cincinnati Bengals,\n,http://www.bengals.com,Cincinnati Bengals : News Stories
3,News from the Cincinnati Bengals,\n,http://www.bengals.com,Cincinnati Bengals : News Stories
4,News from the Cincinnati Bengals,\n,http://www.bengals.com,Cincinnati Bengals : News Stories


In [7]:
xml_data

'<?xml version="1.0" encoding="ISO-8859-1"?>\r\n<rss xmlns:dc="http://purl.org/dc/elements/1.1/" version="2.0">\r\n  <channel>\r\n    <title>Cincinnati Bengals : News Stories</title>\r\n    <link>http://www.bengals.com</link>\r\n    <description>News from the Cincinnati Bengals</description>\r\n    <item>\r\n      <title>Mixon, Bengals hit ground running</title>\r\n      <link>http://www.bengals.com/news/article-1/Mixon-Bengals-hit-ground-running-/5ace91e4-b957-4aeb-a359-9cdea27789ff</link>\r\n      <description>Friday was Joe Mixon\'s NFL debut. Joe Mixon had Trey Hopkins with the chocolate covered pretzels. Mixon, the Bengals rookie running back, got a birthday care package from his mother on the first d...</description>\r\n      <pubDate>Sat, 12 Aug 2017 13:02:19 GMT</pubDate>\r\n      <guid>http://www.bengals.com/news/article-1/Mixon-Bengals-hit-ground-running-/5ace91e4-b957-4aeb-a359-9cdea27789ff</guid>\r\n      <dc:creator>Geoff Hobson</dc:creator>\r\n      <dc:date>2017-08-12T13

In [8]:
root = ET.fromstring(xml_data)

In [9]:
root

<Element 'rss' at 0x0000026F7E1837C8>

In [22]:
all_records = []
for i, child in enumerate(root):
    record = {}
    for subchild in child:
        record[subchild.tag] = subchild.text
        all_records.append(record)

In [11]:
root.tag


'rss'

In [15]:
root.text

'\n  '

In [13]:
root.attrib

{'version': '2.0'}

In [14]:
for child in root:
    print(child.tag, child.attrib)

channel {}


In [19]:
def getTitle(item):
    for child in item:
        if child.tag == 'title':
            return child.text

def getItems(root):
    itemRecords = []
    for child in root:
        if child.tag == 'item':
            itemRecords.append(getTitle(child))
        else:
            itemRecords.extend(getItems(child))
    return itemRecords
            
            

In [20]:
getItems(root)

['Mixon, Bengals hit ground running',
 'Driskel legs Bengals past Bucs',
 'Buccaneers at Bengals Postgame Quotes',
 'Quick hits: Mom, McCoy surprise Mixon; Dalton upbeat',
 'Bengals flex depth, Mixon in opening win',
 'Bengals vs. Buccaneers Game Notes',
 "For openers, it's all relative",
 "Hobson's Choice Podcasts: John Stofa",
 'Fan Focus: Thanks Art Modell',
 'Bengals hope kicking game reveals speed and depth',
 'Bengals Debut New Food Lineup and Other Enhancements at PBS',
 "Camp report: Lewis trying to keep 'stronger' roster healthy as Mixon, Lawson ready for debuts",
 'Marvin Lewis News Conference Transcript 8/9',
 'Early hits: With no restrictions, Lewis eyes Bucs; McCarron shrugs off rumors again',
 'Camp report: history and hits']

In [22]:
api_url = 'http://www.49ers.com/cda-web/rss-module.htm?tagName=Insider%20Columnists'

session = requests.Session()
session.mount('http://', requests.adapters.HTTPAdapter(max_retries=3))
raw_data = session.get(api_url)
xml_data = raw_data.text
root = ET.fromstring(xml_data)
getItems(root)

['On the Road with Rod: 2010 Recap',
 'Points of the Game: Cardinals',
 'Points of the Game: Rams',
 'Points of the Game: Chargers',
 "Brunner's Blog: Bowl Season",
 'Points of the Game: Seahawks',
 'Points of the Game: Packers',
 'Points of the Game: Cardinals',
 'Points of the Game: Buccaneers',
 'Eye on the Midwest: Trip to Wrigley',
 'Points of the Game: Rams',
 'On the Road With Rod: London',
 'Points of the Game: Broncos',
 'Points of the Game: Panthers',
 'Points of the Game: Raiders']

In [24]:
api_url = 'http://www.patriots.com/rss/article/mobile/all/News%20-%20Mobile'

session = requests.Session()
session.mount('http://', requests.adapters.HTTPAdapter(max_retries=3))
raw_data = session.get(api_url)
xml_data = raw_data.text
root = ET.fromstring(xml_data)
getItems(root)

['Bill Belichick Conference Call Transcript 8/11',
 'Patriots Sign Rookie DL Keionta Davis; Release Rookie DB Dwayne Thomas',
 "Tom Brady's family surprised him at training camp",
 'TuneIn Announces Live Joint Audiocast with the Patriots and Texans from Texans Training Camp',
 'Top 5 Photos from Jaguars at Patriots presented by CarMax',
 "Here's how your Patriots rank on Madden '18",
 'News Blitz 8/11: Reaction to Jags game',
 'New England Patriots Training Camp Schedule and Guide',
 'Stock Watch: Garoppolo shines, defense struggles',
 'Analysis: Up and down night for rookies',
 'Random Thoughts: Big plays lead Jags over Pats',
 'Game Notes: Garoppolo makes the start',
 'Patriots fall to Jags 31-24 in opener',
 'Gamebook: Full Jaguars vs. Patriots Stats',
 'Jacksonville Jaguars Postgame Transcripts 8/10']

In [25]:
api_url = 'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News'

session = requests.Session()
session.mount('http://', requests.adapters.HTTPAdapter(max_retries=3))
raw_data = session.get(api_url)
xml_data = raw_data.text
root = ET.fromstring(xml_data)
getItems(root)

['News and Notes 8/12: Austin Howard Practices, Ravens Shuffle Offensive Line',
 'Ravens Training Camp: Saturday Transcript',
 "Justin Tucker Shows He's Human, Then Reverts Back to a Field-Goal Kicking Robot",
 "Ravens Rookie Tim White's Season Ends Just When It Was Heating up",
 "Joe Flacco Won't Play in Preseason Games, But Expected to Be Ready Week 1",
 'Ravens Guard Alex Lewis Out for the Year With Shoulder Injury',
 'Injury Updates on Joe Flacco, Alex Lewis, Tim White',
 'Undrafted Rookies Impress With Big Plays on Bigger Stage',
 "Late For Work 8/11: We Know, We Know. It's One Preseason Game. But That Ravens Defense Looked GOOD",
 "Ryan Mallett Plays 'Winning Football,' But Wants to Be More Consistent",
 'Transcripts: Ravens Halftime Quotes',
 'Transcripts: Redskins Postgame Quotes',
 'Transcripts: Ravens Postgame Quotes',
 'Offensive Tackle Austin Howard Itching to Practice, Hopes to Return Soon',
 'Game Recap: Ravens 23, Redskins 3']